In [1]:
import pandas as pd
from pymongoarrow.api import find_pandas_all
import sys
sys.path.insert(1, '/Users/peteberryman/Desktop/bts-shiny')
from data import __DB__

View relationship between sprint speed, batter handedness and HP-to-1B:

https://charts.mongodb.com/charts-beat-the-streak-cejqw/dashboards/5d97b380-6263-41e0-945c-58f32d00b379

In [2]:
sprint_speeds_df: pd.DataFrame = find_pandas_all(__DB__.sprintSpeeds, dict(), projection = {'_id': False})
sprint_speeds_df

,batter,hp_to_1b,speed,year,bats
0,514917,4.18,29.9,2015,S
1,453211,4.14,29.8,2015,R
2,592261,4.07,30.6,2015,R
3,606299,NaN,29.1,2015,R
4,608671,4.13,29.9,2015,L
...,...,...,...,...,...
5190,425877,4.91,22.8,2018,R
5191,435263,4.98,22.7,2018,L
5192,467092,4.93,22.7,2018,R
5193,572180,NaN,22.5,2018,R


In [3]:
sprint_speeds_df['speed'].isna().value_counts()

speed
False    5195
Name: count, dtype: int64

In [4]:
sprint_speeds_df['hp_to_1b'].isna().value_counts()

hp_to_1b
False    4267
True      928
Name: count, dtype: int64

In [5]:
sprint_speeds_df['bats'].value_counts()

bats
R    2892
L    1683
S     620
Name: count, dtype: int64

Assuming switch-hitters have their best HP-to-1B time as a lefty

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Dummy variable for batter handedness
sprint_speeds_df['rhb'] = sprint_speeds_df['bats'] == 'R'

speed_df_no_na = sprint_speeds_df.dropna()
speed_reg = LinearRegression().fit(speed_df_no_na[['rhb', 'speed']], speed_df_no_na['hp_to_1b'])
print('R^2:', round(float(r2_score(speed_df_no_na['hp_to_1b'], speed_reg.predict(speed_df_no_na[['rhb', 'speed']]))), 2))

sprint_speeds_df['pred_hp_to_1b'] = speed_reg.predict(sprint_speeds_df[['rhb', 'speed']]).round(2)
sprint_speeds_df['hp_to_1b'] = sprint_speeds_df.apply(lambda row: row['hp_to_1b'] if row['hp_to_1b'] > 0 else row['pred_hp_to_1b'], axis = 1)
sprint_speeds_df.drop(['rhb'], axis = 1, inplace = True)
sprint_speeds_df

R^2: 0.83


,batter,hp_to_1b,speed,year,bats,pred_hp_to_1b
0,514917,4.18,29.9,2015,S,4.02
1,453211,4.14,29.8,2015,R,4.14
2,592261,4.07,30.6,2015,R,4.04
3,606299,4.23,29.1,2015,R,4.23
4,608671,4.13,29.9,2015,L,4.02
...,...,...,...,...,...,...
5190,425877,4.91,22.8,2018,R,5.06
5191,435263,4.98,22.7,2018,L,4.97
5192,467092,4.93,22.7,2018,R,5.07
5193,572180,5.10,22.5,2018,R,5.10


Save model as pickle

In [7]:
import pickle

with open('hp_to_1b.pkl', 'wb') as pkl:
    pickle.dump(speed_reg, pkl)